In [115]:
import numpy as np
import pandas as pd 
from keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
from PIL import Image
import cv2
import glob
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization


In [116]:
PATH='C:/Users/khaou/Downloads/dogs-vs-cats1/dogs-vs-cats'

In [117]:
files= os.listdir("C:/Users/khaou/Downloads/dogs-vs-cats1/dogs-vs-cats/train/")

In [118]:
cat_train = [fn for fn in files if 'cat' in fn]
dog_train = [fn for fn in files if 'dog' in fn]


In [119]:
len(cat_train), len(dog_train)

(11850, 11850)

In [120]:
files1= os.listdir("C:/Users/khaou/Downloads/dogs-vs-cats1/dogs-vs-cats/valid/cat/")
files2= os.listdir("C:/Users/khaou/Downloads/dogs-vs-cats1/dogs-vs-cats/valid/dog/")

In [121]:
cat_val = [fn for fn in files1 if 'cat' in fn]
dog_val = [fn for fn in files2 if 'dog' in fn]
len(cat_val), len(dog_val)


(100, 100)

In [122]:
print(cat_val[1])

cat.10118.jpg


In [123]:
def img_to_array(img, data_format='channels_last', dtype='float32'):
    """Converts a PIL Image instance to a Numpy array.
    # Arguments
        img: PIL Image instance.
        data_format: Image data format,
            either "channels_first" or "channels_last".
        dtype: Dtype to use for the returned array.
    # Returns
        A 3D Numpy array.
    # Raises
        ValueError: if invalid `img` or `data_format` is passed.
    """
    if data_format not in {'channels_first', 'channels_last'}:
        raise ValueError('Unknown data_format: %s' % data_format)
    # Numpy array x has format (height, width, channel)
    # or (channel, height, width)
    # but original PIL image has format (width, height, channel)
    x = np.asarray(img, dtype=dtype)
    if len(x.shape) == 3:
        if data_format == 'channels_first':
            x = x.transpose(2, 0, 1)
    elif len(x.shape) == 2:
        if data_format == 'channels_first':
            x = x.reshape((1, x.shape[0], x.shape[1]))
        else:
            x = x.reshape((x.shape[0], x.shape[1], 1))
    else:
        raise ValueError('Unsupported image shape: %s' % (x.shape,))
    return x


In [124]:
train_files = np.concatenate([cat_train[:2000], dog_train[:2000]]) # For technical issues we will only use 2000 images for each class
validation_files = np.concatenate([cat_val, dog_val])

In [125]:
train_files[1]

'cat.1.jpg'

In [126]:
validation_files[1]

'cat.10118.jpg'

In [127]:
train_imgs = [img_to_array(load_img('C:/Users/khaou/Downloads/dogs-vs-cats1/dogs-vs-cats/train/'+img, target_size=IMG_DIM)) for img in train_files]

In [128]:
print(train_imgs[1])

[[[ 39.  44.  40.]
  [ 40.  44.  43.]
  [ 41.  45.  46.]
  ...
  [210. 209. 181.]
  [207. 204. 171.]
  [201. 199. 161.]]

 [[ 40.  45.  41.]
  [ 40.  44.  43.]
  [ 41.  45.  46.]
  ...
  [207. 203. 176.]
  [203. 200. 169.]
  [197. 195. 157.]]

 [[ 39.  44.  40.]
  [ 38.  42.  41.]
  [ 37.  41.  42.]
  ...
  [195. 191. 166.]
  [198. 193. 164.]
  [205. 200. 168.]]

 ...

 [[ 29.  27.  28.]
  [ 25.  23.  24.]
  [ 22.  20.  21.]
  ...
  [ 50.  37.  31.]
  [ 41.  28.  22.]
  [ 49.  38.  32.]]

 [[ 32.  30.  31.]
  [ 26.  24.  25.]
  [ 22.  20.  21.]
  ...
  [ 44.  31.  23.]
  [ 42.  29.  21.]
  [ 55.  45.  36.]]

 [[ 32.  30.  31.]
  [ 25.  23.  24.]
  [ 21.  19.  20.]
  ...
  [ 59.  46.  38.]
  [ 51.  38.  30.]
  [ 40.  30.  21.]]]


In [ ]:
train_imgs = np.array(train_imgs)

In [ ]:
train_imgs[1]

In [ ]:
filenames =list(train_files)
categories_train = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories_train.append(1)
    else:
        categories_train.append(0)


In [ ]:
print(categories_train)

In [ ]:
validation_imgs_cat = [img_to_array(load_img('C:/Users/khaou/Downloads/dogs-vs-cats1/dogs-vs-cats/valid/cat/'+img, target_size=IMG_DIM)) for img in cat_val]
validation_imgs_cat = np.array(validation_imgs_cat)


In [ ]:
validation_imgs_cat.shape

In [ ]:
val_leb_cat=[0] * 100

In [ ]:
print(val_leb_cat)

In [ ]:
validation_imgs_dog = [img_to_array(load_img('C:/Users/khaou/Downloads/dogs-vs-cats1/dogs-vs-cats/valid/dog/'+img, target_size=IMG_DIM)) for img in dog_val]
validation_imgs_dog = np.array(validation_imgs_dog)


In [ ]:
validation_imgs_dog.shape

In [ ]:
val_leb_dog=[1] * 100

In [100]:
validation_imgs = np.concatenate((validation_imgs_cat, validation_imgs_dog))

In [102]:
validation_imgs.shape

(200, 224, 224, 3)

In [103]:
validation_leb = np.concatenate((val_leb_cat, val_leb_dog))

In [105]:
print('Train dataset shape:', train_imgs.shape, 
      '\nValidation dataset shape:', validation_imgs.shape)

Train dataset shape: (4000, 224, 224, 3) 
Validation dataset shape: (200, 224, 224, 3)


In [106]:
train_set = ImageDataGenerator(rescale=1./255).flow(train_imgs, categories_train, batch_size=30)

val_set = ImageDataGenerator(rescale=1./255).flow(validation_imgs, validation_leb, batch_size=20)

In [107]:

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


In [108]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 112, 112, 32)     0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 401408)            0         
                                                                 
 dense_2 (Dense)             (None, 128)               51380352  
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 51,381,377
Trainable params: 51,381,377
Non-trainable params: 0
__________________________________________

In [111]:
model.compile( loss='binary_crossentropy', metrics=['accuracy'])

In [112]:
from keras.callbacks import ModelCheckpoint
checkpointer1 = ModelCheckpoint(filepath='weight12.hdf5', verbose=1, save_best_only=True)

In [114]:
history = model.fit_generator(train_set, steps_per_epoch=len(train_set),
validation_data=val_set, validation_steps=len(val_set), epochs=20, verbose=1)

C:\Users\khaou\AppData\Local\Temp/ipykernel_7884/4012066420.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_set, steps_per_epoch=len(train_set),


Epoch 1/20
134/134 [==============================] - 127s 897ms/step - loss: 4.8490 - accuracy: 0.5725 - val_loss: 0.5942 - val_accuracy: 0.6500
Epoch 2/20
134/134 [==============================] - 99s 742ms/step - loss: 0.6040 - accuracy: 0.6965 - val_loss: 0.5409 - val_accuracy: 0.6900
Epoch 3/20
134/134 [==============================] - 101s 754ms/step - loss: 0.4174 - accuracy: 0.8165 - val_loss: 0.5661 - val_accuracy: 0.6950
Epoch 4/20
134/134 [==============================] - 101s 754ms/step - loss: 0.2865 - accuracy: 0.8892 - val_loss: 0.5926 - val_accuracy: 0.7200
Epoch 5/20
134/134 [==============================] - 97s 725ms/step - loss: 0.1793 - accuracy: 0.9463 - val_loss: 0.6468 - val_accuracy: 0.7350
Epoch 6/20
134/134 [==============================] - 99s 740ms/step - loss: 0.1076 - accuracy: 0.9675 - val_loss: 0.7428 - val_accuracy: 0.7150
Epoch 7/20
134/134 [==============================] - 98s 729ms/step - loss: 0.0661 - accuracy: 0.9818 - val_loss: 0.8592 - val

KeyboardInterrupt: 

In [ ]:
metrics = model.evaluate(X_test,  y_test, verbose=1)
print("Test Accuracy: {}".format(metrics[1]))
print("Test Loss: {}".format(metrics[0]))

In [ ]:
# Plot the loss and accuracy curves for training and validation 
plt.plot(history.history['val_loss'], color='b', label="validation loss")
plt.title("Test Loss")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()